In [6]:
import os, shutil,pathlib,exifread
r"""Sorting out the raw images collected from camera trap data
Loop through each location and subsequent feed type and images by extracting
the original date the image taken using exifread."""

"""
/test_data
    - C2
    - C3
            - fat_balls
        - standard_seed
        - sunflower_seed
            - date ## This folder is created in this cell
                - 2022_11_23 ## images capture date is extracted saving the image into a seperate folder into the date/raw_images folder
                    - cropped_images ## Images are then cropped using megadetector
                    - raw_images ## This folder temporarily stores raw images before they are cropped by megadetector
            - raw_images ## Contains the folders of raw images
                - 100EK113 ## Images from camera traps are stored in this folder
    - C4
"""
# Define a working directory pathway 
cwd = pathlib.Path("test_data")

# Loop through each folder within the working direcory
for location in os.listdir(cwd):
    
    # Define new file pathway.  
    location_dir = cwd/location 
    
    # Loop through next set of folders 
    for feed_type in os.listdir(location_dir):
        
        # Define new folder paths to be used 
        feed_type_dir = location_dir/feed_type
        image_folder_dir = feed_type_dir/"raw_images"
        
        # Loop though each of the image
        for image in os.listdir(image_folder_dir):
                
            # Corrupted files or invalid file types to be passed
            try: 
                """Using try/except will pass through corrupt images or files that
                do not have an exif data to extract"""
                    
                # Create a image file path to the camera trap image 
                exif_file = str(feed_type_dir/"raw_images"/image) 
                img_file = open(exif_file,"rb")
                tags = exifread.process_file(img_file)
                """Extract the tags embedded within the image, in this case the Date and time of original image, for this                     scenario we are only looking for the date so we can extract from within a string just the date and save 
                    this into a new variable"""
                    
                date_of_image = str(tags["EXIF DateTimeOriginal"])
                day_of_image = str(date_of_image[0:10])
                    
                #Replace the : with _ as it will be easier to open at a later point.
                date = day_of_image.replace(":","_") 
                
                # Define the new file location path for each of the images to be copied to
                newpath1 = feed_type_dir/"date"/date/"raw_images"
                img_file.close()
                    
                """Check if there is a directory already set up in the folder directory, if there isn't a new folder will be
                made. The image is then checked to prevent any duplication if it has already been copied before"""
                if not os.path.exists(newpath1):
                    os.makedirs(newpath1)
                dated_image_path = newpath1/image
                if not os.path.exists(dated_image_path):
                    shutil.copy(exif_file,dated_image_path)
            except:
                pass       

In [7]:
# Loop through the directory and use megadetector to detect animals and crop bounding boxes to save into a new directory. 
import os, shutil,pathlib,exifread

# Define the root file pathway 
cwd = pathlib.Path("test_data")

#After each loop a the new folder path is saved into a new variable to make looping more tidy
for location in os.listdir(cwd):
    
    # Define new folder pathway to loop through 
    feed_type_dir = cwd/location
    for feed_type in os.listdir(feed_type_dir):
        
        # Define file pathway for folders for each date
        date_dir = feed_type_dir/feed_type/"date"
        
        # Loop through the dates folders 
        for date in os.listdir(date_dir):
            try:
                ## Define the arguments needed to pass into run_batch_detector and the subsequent crop_detections,
                # the required arguments can be found in the comments inside the files.
                raw_images_dir = str(os.path.abspath(date_dir/date/"raw_images"))
                output_file_path = str(os.path.abspath(date_dir/date/"run_batch_detector_output.json"))
                cropped_image_output_dir = str(os.path.abspath(date_dir/date/"cropped_images"))
                
                # Check if a directory for cropped_image_output has already been created and create one if not 
                if not os.path.exists(cropped_image_output_dir):
                    os.makedirs(cropped_image_output_dir)
                    
                # Check if a directory for .json logs files has already been created and create one if not
                log_dir = os.path.abspath(pathlib.Path("crop_log"))
                if not os.path.exists(log_dir):
                    os.makedirs(log_dir)
                    
                """Run run_batch_detector.py with the args: --megadetector file location, --images directory, --output directory for .json files"""
                %run "C:/git/cameratraps/detection/run_detector_batch.py" "C:/megadetector/md_v5a.0.0.pt" "$raw_images_dir" "$output_file_path" --recursive --output_relative_filenames --threshold 0.8
                
                """Run crop_detections.py with the args: --.json file directory, --image output directory"""
                %run "c:/git/cameratraps/classification/crop_detections.py" "$output_file_path" "$cropped_image_output_dir" --images-dir "$raw_images_dir" --square-crops --logdir "$log_dir"
                
                # Delete the .json file storing the bounding box data to reduce file cluttering
                os.remove(output_file_path)
                
                # remove raw images in the dates folder to reduce directory memory size
                for image in os.listdir(raw_images_dir):
                    
                    # Define the file path for the raw image
                    dirs = os.path.abspath(pathlib.Path(date_dir/date/"raw_images"/image))
                    
                    # Delete the file
                    os.remove(dirs)
                    
                    # Print which file has been deleted
                    print(image, " Deleted")
            except:
                pass
            print("")
        print("")
    print("")

1 image files found in the input directory


C:\Users\vince\anaconda3\envs\cameratraps-detector\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch reports 1 available CUDA devices
GPU available: True
Using PyTorch version 1.10.1


INFO:yolov5:Fusing layers... 
Fusing layers... 
INFO:yolov5:Model summary: 574 layers, 139990096 parameters, 0 gradients
Model summary: 574 layers, 139990096 parameters, 0 gradients


Sending model to GPU
Loaded model in 4.95 seconds
Loaded model in 4.95 seconds


...

Finished inference for 1 images in 1.11 seconds
Output file saved at C:\Users\vince\Documents\BIO3199\Vincent's_Project\Python\test_data\C4\sunflower_seed\date\2022_12_21\run_batch_detector_output.json
Done!
Getting bbox info for 1 images...


100%|██████████| 1/1 [00:00<00:00, 1038.71it/s]


Reading/downloading 1 images and cropping...


100%|██████████| 1/1 [00:00<00:00, 10.66it/s]

Downloaded 0 images.
Made 1 new crops.
0 images failed to download or crop.
12210326.JPG  Deleted



